In [13]:
import csv
from bs4 import BeautifulSoup
import requests

# Linkleri çekme
url = "http://anitsayac.com/#document"
link_basi = "http://anitsayac.com/"
linkler = []

response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')

for div in soup.find_all('span', attrs={'class': 'xxy'}):
    link = div.find('a')['href']
    link_tamami = link_basi + link
    linkler.append(link_tamami)

# CSV dosyasını açma ve başlık yazma
with open("veriler.csv", mode="w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ["Name", "Age", "City", "Date", "Why Was She/He Killed", 
                  "Who Was Killed By", "Request for Protection", "The Way Was Killed", 
                  "Status of the Suspect"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for full_link in linkler:
        # Web sayfasını isteği yapma ve içeriği alma
        response = requests.get(full_link)
        html_content = response.text
        # BeautifulSoup nesnesini oluşturma
        soup = BeautifulSoup(html_content, 'html.parser')

        data = {}

        # <body> etiketini bulup içeriği çekme
        body_tag = soup.find('body')
        if body_tag:
            text = body_tag.get_text(separator="\n")  # Her etiket arasına yeni satır ekleyerek içeriği al
            
            # Verileri çekme ve düzenleme
            lines = text.split("\n")
            label = None
            for line in lines:
                if ":" in line:
                    label, value = line.split(":", 1)
                    label = label.strip()
                    value = value.strip()
                    data[label] = value
                elif label is not None:
                    data[label] += " " + line.strip()
        
        # Verileri CSV dosyasına yazma
        writer.writerow({
            "Name": data.get("Ad Soyad", ""),
            "Age": data.get("Maktülün yaşı", ""),
            "City": data.get("İl/ilçe", ""),
            "Date": data.get("Tarih", ""),
            "Why Was She/He Killed": data.get("Neden öldürüldü", ""),
            "Who Was Killed By": data.get("Kim tarafından öldürüldü", ""),
            "Request for Protection": data.get("Korunma talebi", ""),
            "The Way Was Killed": data.get("Öldürülme şekli", ""),
            "Status of the Suspect": data.get("Failin durumu", ""),
        })
